In [1]:
import geopandas as gpd
import shapely  

# Path:
path = "/workspaces/Satelite/data/aargau.gpkg"
file = gpd.read_file(path)

In [9]:
import geopandas as gpd
from shapely.geometry import Polygon

def simplify_and_filter_geometries(gdf, simplify_tolerance=5, min_area=5000):
    """
    Simplifies the geometries in a GeoDataFrame and removes geometries smaller than a specified area.

    Args:
    gdf (GeoDataFrame): The GeoDataFrame containing the geometries to process.
    simplify_tolerance (float): The tolerance for simplifying the geometries, with a smaller
                                number preserving more detail.
    min_area (float): The minimum area in square meters for geometries to be retained.

    Returns:
    GeoDataFrame: A new GeoDataFrame with the simplified and filtered geometries.
    """
    # Simplify geometries
    gdf['geometry'] = gdf['geometry'].simplify(tolerance=simplify_tolerance, preserve_topology=True)
    
    # Filter out small geometries
    gdf = gdf[gdf['geometry'].area >= min_area]

    return gdf


In [10]:
import geopandas as gpd
import numpy as np

def filter_hose_like_shapes(gdf, max_aspect_ratio=5, min_compactness=0.1):
    """
    Filters out elongated and non-compact geometries from a GeoDataFrame.

    Args:
    gdf (GeoDataFrame): GeoDataFrame with the geometries to filter.
    max_aspect_ratio (float): Maximum allowed aspect ratio.
    min_compactness (float): Minimum allowed compactness.

    Returns:
    GeoDataFrame: Filtered GeoDataFrame.
    """
    def calculate_aspect_ratio(geometry):
        minx, miny, maxx, maxy = geometry.bounds
        return (maxx - minx) / (maxy - miny) if (maxy - miny) != 0 else np.inf

    def calculate_compactness(geometry):
        area = geometry.area
        perimeter = geometry.length
        return 4 * np.pi * area / (perimeter ** 2) if perimeter != 0 else 0

    # Calculate aspect ratio and compactness
    gdf['aspect_ratio'] = gdf['geometry'].apply(calculate_aspect_ratio)
    gdf['compactness'] = gdf['geometry'].apply(calculate_compactness)

    # Apply filters
    filtered_gdf = gdf[
        (gdf['aspect_ratio'] <= max_aspect_ratio) & 
        (gdf['compactness'] >= min_compactness)
    ]

    # Drop auxiliary columns
    filtered_gdf = filtered_gdf.drop(columns=['aspect_ratio', 'compactness'])

    return filtered_gdf

# Example usage
path = "/workspaces/Satelite/data/aargau.gpkg"
file = gpd.read_file(path)
processed_file = simplify_and_filter_geometries(file)
processed_file = filter_hose_like_shapes(processed_file)

/home/tfuser/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/tfuser/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [19]:
import geopandas as gpd
import numpy as np

def simplify_and_filter_geometries(gdf, simplify_tolerance=5, min_area=5000):
    """
    Simplifies the geometries in a GeoDataFrame and removes geometries smaller than a specified area.

    Args:
    gdf (GeoDataFrame): The GeoDataFrame containing the geometries to process.
    simplify_tolerance (float): The tolerance for simplifying the geometries.
    min_area (float): The minimum area in square meters for geometries to be retained.

    Returns:
    GeoDataFrame: A new GeoDataFrame with the simplified and filtered geometries.
    """
    # Create a copy to ensure we are not modifying a view
    gdf = gdf.copy()

    # Simplify geometries
    gdf['geometry'] = gdf['geometry'].simplify(tolerance=simplify_tolerance, preserve_topology=True)
    
    # Filter out small geometries using loc to avoid SettingWithCopyWarning
    gdf = gdf.loc[gdf['geometry'].area >= min_area]

    return gdf

# Usage
path = "/workspaces/Satelite/data/aargau.gpkg"
file = gpd.read_file(path)
processed_file = simplify_and_filter_geometries(file)


In [20]:
# Save the result
processed_file.to_file("/workspaces/Satelite/data/aargau_processed.gpkg", driver='GPKG')
